<a href="https://colab.research.google.com/github/thejjenkins/ComputerScience2/blob/main/v2_imageCNNipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is the skeleton of any neural network I want to build. 

It is a template. This is the template found at the website:
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Some additional websites I used:
* https://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html
* https://pytorch.org/docs/stable/generated/torch.argmax.html

The linear functions will be changed out for conv2D (maybe conv3D) functions.

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define the class
class NeuralNetwork(nn.Module):
    # init is the definition while forward initiates movement through the defined layers
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # layer 1
            nn.ReLU(),
            nn.Linear(512, 512), # layer 2
            nn.ReLU(),
            nn.Linear(512, 10), # layer 3
        )

    # forward is the movement through the network
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits # returns a bracket containing a bracket with 10 individual float numbers ([0, 1, 10])

model = NeuralNetwork().to(device) # instantiate network
print(model) # prints an outline of the model

X = torch.rand(1, 28, 28, device=device) # mnist photo of Xshape = print(X.shape) = torch.Size([0, 1, 28, 28])
logits = model(X) # ([0, 1, 10])
print(logits) 
print(f"First dimension of logits: {logits[0][:]}")
pred_probab = nn.Softmax(dim=1)(logits) # softmax = Sigmoid( on logits[0][:] ))
print(f"logits after sigmoid: {pred_probab[0][:]}")
y_pred = pred_probab.argmax(1) # returns the index of the largest number (highest probability) after sigmoid operation
print(f"Predicted class: {y_pred}") # y_pred = index of highest probability. Therefore, the predicted label. 

input_image = torch.rand(3,28,28) # 3 color channel 28x28 pixel image ([0, 3, 28, 28])
print(input_image.size()) # torch.Size([3, 28, 28]) # the zeroth dimension is hidden so its actually ([0, 3, 28, 28])

flatten = nn.Flatten() # instantiates Flatten subclass of nn without calling model(X) as in line 39
flat_image = flatten(input_image) # reduces input_image to a bracket containing a bracket of 784 individual float numbers ([0, 1, 784]) instead of a bracket containing 28 individual brackets each containing 28 individual float numbers ([0, 1, 28, 28])
print(flat_image.size()) # torch.Size([3, 784]) # the zeroth dimension is hidden so its actually ([0, 3, 784])

layer1 = nn.Linear(in_features=28*28, out_features=20) # instantiates layer 1 without calling model(X) as in line 39
hidden1 = layer1(flat_image) # this is the input layer. returns a bracket containing 3 individual brackets each containing 20 individual float numbers ([0, 3, 20])
print(hidden1.size()) # torch.Size([3, 20]) # the zeroth dimension is hidden so its actually ([0, 3, 784])

# Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.
# In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.
# James: The non-linear activation might produce a heat-map
print(f"Before ReLU: {hidden1}\n\n") # prints the torch.Size([0, 3, 20])
hidden1 = nn.ReLU()(hidden1) # every negative float is set to 0 and every positive float is untouched
print(f"After ReLU: {hidden1}") # prints the ([0, 3, 20]) tensor

# nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.
seq_modules = nn.Sequential( # This seq_model is essentially the ``forward`` function of a class
    flatten,   # These have already been defined.
    layer1,    # This order can be used as a template. 
    nn.ReLU(), 
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28) # input_image.shape == ([0, 3, 28, 28])
logits = seq_modules(input_image) # logits = ([0, 3, 10])
print(f"Logits: {logits}")

softmax = nn.Softmax(dim=1) # sigmoid
pred_probab = softmax(logits) # sigmoid on the ([0, 3, 10])
print(f"Pred_probab = {pred_probab}")

print(f"Model structure: {model}\n\n") # Prints an outline of the model
for name, param in model.named_parameters(): # prints the weights tensor and bias tensor for each layer in the model
  print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
tensor([[-0.1156, -0.0701,  0.0552, -0.0184, -0.1279,  0.0042,  0.0519,  0.0109,
          0.0482,  0.0641]], device='cuda:0', grad_fn=<AddmmBackward0>)
First dimension of logits: tensor([-0.1156, -0.0701,  0.0552, -0.0184, -0.1279,  0.0042,  0.0519,  0.0109,
         0.0482,  0.0641], device='cuda:0', grad_fn=<SliceBackward0>)
logits after sigmoid: tensor([0.0898, 0.0939, 0.1065, 0.0989, 0.0887, 0.1012, 0.1061, 0.1019, 0.1057,
        0.1074], device='cuda:0', grad_fn=<SliceBackward0>)
Predicted class: tensor([9], device='cuda:0')
torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[-0.6966, -0.2612,  0.0740,  0.0561,  0

In [ ]:
img = torchvision.io.read_image("/content/b1resized.png")
img=img.unsqueeze(dim=1)
_,channel,width,height=img.shape
img.shape
flatten_img = nn.Flatten()
flatten_img = flatten_img(img)
flatten_img.shape

torch.Size([1, 307200])

In [ ]:
Model = model()
Model.parameters()

<generator object Module.parameters at 0x7feed3141c80>